In [1]:
import soccerdata as sd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

[12/16/23 00:57:35] INFO     No custom team name replacements found. You can configure these in       ]8;id=286462;file://c:\Python\Python310\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=213771;file://c:\Python\Python310\lib\site-packages\soccerdata\_config.py#84\84]8;;\
                             C:\Users\SalahEddine.G\soccerdata\config\teamname_replacements.json.                  

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=11988;file://c:\Python\Python310\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=828489;file://c:\Python\Python310\lib\site-packages\soccerdata\_config.py#162\162]8;;\
                             C:\Users\SalahEddine.G\soccerdata\config\league_dict.json.                            

In [2]:
elo = sd.ClubElo()

[12/16/23 00:57:38] INFO     Saving cached data to C:\Users\SalahEddine.G\soccerdata\data\ClubElo     ]8;id=861629;file://c:\Python\Python310\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=936763;file://c:\Python\Python310\lib\site-packages\soccerdata\_common.py#91\91]8;;\

In [3]:
leagues = elo.available_leagues()
leagues

['ENG-Premier League',
 'ESP-La Liga',
 'FRA-Ligue 1',
 'GER-Bundesliga',
 'ITA-Serie A']

We want to loop in each week of the season marking the elo of each team by rounds. <br>
The rounds are generally weeks starting from the first week of august and ends at the last week of june 

In [4]:
from datetime import datetime, timedelta
ClubsElos = pd.DataFrame()

def week_num(date):
    week, year, _ = date.isocalendar()
    if date.month == 1 and week >= 52:
        year -= 1
    return f"Week {week}: {date}"

# Loop through years
for year in range(2010, 2023):
    # Set start date
    start_date = datetime(year, 8, 7)

    # Loop until June 30th, 2023
    while start_date <= datetime(2023, 7, 30) and (start_date.month < 7 or start_date.month > 7):
        # Print week number and date
        elos = elo.read_by_date(f"{start_date.year}-{start_date.month}-{start_date.day}")
        # Filter by the 5 big leagues
        filtered_seasons = elos[elos.league.isin(leagues)]
        # Correct the ranking of the teams
        filtered_seasons['rank'] = range(1, len(filtered_seasons) + 1)
        # Add the week colomn 
        filtered_seasons['week'] = f"{start_date.year}-{start_date.month}-{start_date.day}" 
        #append the data to the df
        ClubsElos = pd.concat([ClubsElos, filtered_seasons], ignore_index=False)

        # Add 7 days
        start_date += timedelta(days=7)


c:\Python\Python310\lib\site-packages\soccerdata\clubelo.py:18: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(
C:\Users\SalahEddine.G\AppData\Local\Temp\ipykernel_2268\4238681039.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_seasons['rank'] = range(1, len(filtered_seasons) + 1)
C:\Users\SalahEddine.G\AppData\Local\Temp\ipykernel_2268\4238681039.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [18]:
ClubsElos.drop(columns=['level', 'from', 'to'], inplace=True)

In [22]:
# Dictionary to store DataFrames for each league
league_dataframes = {}

# Group by 'league' column and iterate over groups
for league, group_df in ClubsElos.groupby('league'):
    # Store the DataFrame in the dictionary with the league name as the key
    league_dataframes[league] = group_df

ENG_elo_df = league_dataframes['ENG-Premier League']
ESP_elo_df = league_dataframes['ESP-La Liga']
FRA_elo_df = league_dataframes['FRA-Ligue 1']
GER_elo_df = league_dataframes['GER-Bundesliga']
ITA_elo_df = league_dataframes['ITA-Serie A']

In [24]:
import os
# Create a directory to store the CSV files in Google Drive

output_directory = 'D:/myfiles/studies/univ/Master/M2/S3/BD/project/Project_Code/ClubsElo_DataCode'
os.makedirs(output_directory, exist_ok=True)

# Iterate through each league and DataFrame in the dictionary
for league, df in league_dataframes.items():
    # Construct the CSV filename
    csv_filename = f"{league.replace('-', '_').replace(' ', '_')}_elo.csv"

    # Save the DataFrame to a CSV file in Google Drive
    df.to_csv(os.path.join(output_directory, csv_filename), index=True)